In [95]:
import serial
import time
import csv
from bs4 import BeautifulSoup
import subprocess

In [19]:
"""
For initialisation of Github SSH connection, follow this guide:
https://docs.github.com/en/free-pro-team@latest/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent

Step 0: Import the libraries that are used in the following scripts by running the first cell

Step 1: Start Python server in terminal via command: $python3 serverlocal.py

Step 2: If the git repository is not yet cloned, run the script from the next cell
    
Step 3: The fifth cell contains the function for reading the serial data stream from the Arduino. 
        In order to read this stream, the Serial Port has to be defined in line 8 of the script.
        If the script is failing to run with the error 'could not open port /dev/cu.usbmodem14301',
        check the port via the Arduino App by looking in Tools > Port.
        Copy the found port into the Python code and run the script.
        This is necessary for updating the index_local.html with the status of all parts.
    
Step 4: When finished with demonstration, run the third cell, with the commit() function. 
        This will update the pages with the number of pump turns to be accessed with the barcodes via the app.

Step 5: If the csv file or the order list is too long, run the fourth cell with the reset function. 
        This will save the current file with the current date and replace it with a clean file.
"""

'\nFor initialisation of Github SSH connection, follow this guide:\nhttps://docs.github.com/en/free-pro-team@latest/github/authenticating-to-github/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent\n\nStep 1: Start Python server in terminal via command: $python3 serverlocal.py\n\nStep 2: If the git repository is not yet cloned, run the script from the next cell\n    \nStep 3: \n    \nStep 4: When finished with demonstration, run the third cell, with the commit() function. \n        This will update the pages with the number of pump turns to be accessed with the barcodes via the app.\n\nStep 5: If any of the files is malfunctioning (e.g. the csv file or the order list is too long), \n        run the fourth cell with the reset function. This will save the current file with the current date and\n'

In [ ]:
working_path = '/Users/kommunizm/Documents/Work/Uni/Technolab/clean_code/lemonadetechnolab.github.io'

def clone():
    try:
         output = subprocess.check_output(['./git_clone.sh'], cwd=working_path, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
         print("Exception on process, rc=", e.returncode, "output=", e.output)
            
clone()


In [96]:
working_path = '/Users/kommunizm/Documents/Work/Uni/Technolab/clean_code/lemonadetechnolab.github.io'

def commit():
    try:
         output = subprocess.check_output(['./git_commit.sh'], cwd=working_path, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
         print("Exception on process, rc=", e.returncode, "output=", e.output)
            
commit()

In [21]:
working_path = '/Users/kommunizm/Documents/Work/Uni/Technolab/clean_code/lemonadetechnolab.github.io'

def reset_csv():
    try:
         output = subprocess.check_output(['./reset_csv.sh'], cwd=working_path, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
         print("Exception on process, rc=", e.returncode, "output=", e.output)
            
reset_csv()

In [ ]:
working_path = '/Users/kommunizm/Documents/Work/Uni/Technolab/clean_code/lemonadetechnolab.github.io'

def reset_order():
    try:
         output = subprocess.check_output(['./reset_order.sh'], cwd=working_path, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
         print("Exception on process, rc=", e.returncode, "output=", e.output)
            
reset_order()

In [94]:
ser = serial.Serial('/dev/cu.usbmodem14101')
ser.flushInput()

bar_pompw = 'barcode_item_pomp_water.html'
bar_pompa = 'barcode_item_pomp_afvalvat.html'
bar_pomps1 = 'barcode_item_pomp_aardbei.html'
bar_pomps2 = 'barcode_item_pomp_sinaasappel.html'
bar_pompl = 'barcode_item_pomp_beker.html'

time_0 = time.time()

def serial_add(f):
    fin = open("status.html", "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    serial_old = soup.find("span", class_= "serial")
    serial_new = f
    data = data.replace(str(serial_old), '<span class="serial">' + str(serial_new) + '</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def pump_add(f):
    fin = open(f, "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    turns = soup.span
    turns_new = str(int(turns.get_text()) + 1)
    data = data.replace(turns.get_text(), turns_new)
    fin.close()
    fin = open(f, "wt")
    fin.write(data)
    fin.close()
    
def pump_update(f):
    fin = open("status.html", "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    turns = soup.find("span", class_= f)
    turns_new = str(int(turns.get_text()) + 1)
    data = data.replace(str(turns), '<span class="'+ f + '">' + turns_new + '</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def level_ok(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">LOW</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    
def level_low(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">LOW</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()

def level_high(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span style="color: green;" class="' + f + '_level">OK</span>', '<span style="color: red;" class="' + f + '_level">HIGH</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def pump_on(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span class="' + f + '" style="color: red;">OFF</span>', '<span class="' + f + '" style="color: green;">ON</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def pump_off(f):
    fin = open("status.html", "rt")
    data = fin.read()
    data = data.replace('<span class="' + f + '" style="color: green;">ON</span>', '<span class="' + f + '" style="color: red;">OFF</span>')
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
def add_order(f):
    f = f[10:len(f)]
    f = f.replace("smaak1", "Orange")
    f = f.replace("smaak2", "Pear")
    current_time = str(time.asctime(time.localtime()))
    fin = open("status.html", "rt")
    data = fin.read()
    soup = BeautifulSoup(data, "html.parser")
    new_tag = soup.new_tag('li')
    new_tag.string = current_time + ": " + f
    soup.ol.insert(-1, new_tag)
    data = str(soup)
    fin.close()
    fin = open("status.html", "wt")
    fin.write(data)
    fin.close()
    
while True:
    current_time = round((time.time() - time_0) / 60, 2)
    ser_line = ser.readline()
    print(ser_line)
    ser_line = ser_line.decode("ISO-8859-1")
    serial_time = str(time.asctime(time.localtime()))
    print(serial_time + " -- " + ser_line)
    serial_add(ser_line)
    try:
        if ser_line.find("smaak1_low") != -1:
            level_low('s1')
        elif ser_line.find("smaak1_ok") != -1:
            level_ok('s1')
        elif ser_line.find("smaak2_low") != -1:
            level_low('s2')
        elif ser_line.find("smaak2_ok") != -1:
            level_ok('s2')
        elif ser_line.find("smaak3_low") != -1:
            level_low('s3')
        elif ser_line.find("smaak3_ok") != -1:
            level_ok('s3')
        elif ser_line.find("water_low") != -1:
            level_low('w')
        elif ser_line.find("water_ok") != -1:
            level_ok('w')
        elif ser_line.find("limo_high") != -1:
            level_high('l')
        elif ser_line.find("limo_ok") != -1:
            level_ok('l')
        elif ser_line.find("turbiditysensor") != -1:
            turb_val = ser_line[ser_line.find(",")+1:ser_line.find(".")]
            with open("datalog_turb.csv","a") as f:
                writer = csv.writer(f,delimiter=",")
                writer.writerow([current_time,turb_val])
        elif ser_line.find("new_order") != -1:
            add_order(ser_line)
        elif ser_line.find("pompw_high") != -1:
            pump_add(bar_pompw)
            pump_update("w_runs")
            pump_on('w_pomp')
        elif ser_line.find("pompw_low") != -1:
            pump_off('w_pomp')
        elif ser_line.find("pompa_high") != -1:
            pump_add(bar_pompa)
            pump_update("a_runs")
            pump_on('a_pomp')
        elif ser_line.find("pompa_low") != -1:
            pump_off('a_pomp')
        elif ser_line.find("pompl_high") != -1:
            pump_add(bar_pompl)
            pump_update("l_runs")
            pump_on('l_pomp')
        elif ser_line.find("pompl_low") != -1:
            pump_off('l_pomp')
        elif ser_line.find("pomps1_high") != -1:
            pump_add(bar_pomps1)
            pump_update("s1_runs")
            pump_on('s1_pomp')
        elif ser_line.find("pomps1_low") != -1:
            pump_off('s1_pomp')
        elif ser_line.find("pomps2_high") != -1:
            pump_add(bar_pomps2)
            pump_update("s2_runs")
            pump_on('s2_pomp')
        elif ser_line.find("pomps2_low") != -1:
            pump_off('s2_pomp')
        elif ser_line.find("pomps3_high") != -1:
            pump_add(bar_pomps3)
            pump_update("s3_runs")
            pump_on('s3_pomp')
        elif ser_line.find("pomps3_low") != -1:
            pump_off('s3_pomp')
    except:
        break

b'Start\r\n'
Mon Nov 30 15:02:52 2020 -- Start

b'\r\n'
Mon Nov 30 15:02:52 2020 -- 

b'Place your order\r\n'
Mon Nov 30 15:02:52 2020 -- Place your order

b'turbiditysensor,49.00\r\n'
Mon Nov 30 15:02:56 2020 -- turbiditysensor,49.00

b'turbiditysensor,49.00\r\n'
Mon Nov 30 15:02:57 2020 -- turbiditysensor,49.00

b'turbiditysensor,49.00\r\n'
Mon Nov 30 15:02:59 2020 -- turbiditysensor,49.00

b'turbiditysensor,52.00\r\n'
Mon Nov 30 15:03:00 2020 -- turbiditysensor,52.00

b'new_order,smaak2,1\r\n'
Mon Nov 30 15:03:00 2020 -- new_order,smaak2,1

b'smaak2 is gekozen\r\n'
Mon Nov 30 15:03:00 2020 -- smaak2 is gekozen

b'With strength: 1.00\r\n'
Mon Nov 30 15:03:00 2020 -- With strength: 1.00

b'1\r\n'
Mon Nov 30 15:03:00 2020 -- 1

b'57 - s2: 948\r\n'
Mon Nov 30 15:03:01 2020 -- 57 - s2: 948

b'Is smaak2 low?\r\n'
Mon Nov 30 15:03:01 2020 -- Is smaak2 low?

b'61 - water: 768\r\n'
Mon Nov 30 15:03:02 2020 -- 61 - water: 768

b'is limo full?\r\n'
Mon Nov 30 15:03:02 2020 -- is limo full?

b'

b'RFID_val =\r\n'
Mon Nov 30 15:05:03 2020 -- RFID_val =

b'1\r\n'
Mon Nov 30 15:05:03 2020 -- 1

b'pompl_low\r\n'
Mon Nov 30 15:05:03 2020 -- pompl_low

b'Allowed!\r\n'
Mon Nov 30 15:05:04 2020 -- Allowed!

b'4\r\n'
Mon Nov 30 15:05:04 2020 -- 4

b'is klein\r\n'
Mon Nov 30 15:05:04 2020 -- is klein

b'RFID_val =\r\n'
Mon Nov 30 15:05:04 2020 -- RFID_val =

b'25000\r\n'
Mon Nov 30 15:05:04 2020 -- 25000

b'132149\r\n'
Mon Nov 30 15:05:04 2020 -- 132149

b'in cycle:5\r\n'
Mon Nov 30 15:05:04 2020 -- in cycle:5

b'RFID_val =\r\n'
Mon Nov 30 15:05:04 2020 -- RFID_val =

b'1\r\n'
Mon Nov 30 15:05:04 2020 -- 1

b'pompl_high\r\n'
Mon Nov 30 15:05:04 2020 -- pompl_high

b'Allowed!\r\n'
Mon Nov 30 15:05:04 2020 -- Allowed!

b'4\r\n'
Mon Nov 30 15:05:04 2020 -- 4

b'is klein\r\n'
Mon Nov 30 15:05:04 2020 -- is klein

b'RFID_val =\r\n'
Mon Nov 30 15:05:04 2020 -- RFID_val =

b'25000\r\n'
Mon Nov 30 15:05:04 2020 -- 25000

b'RFID_val =\r\n'
Mon Nov 30 15:05:07 2020 -- RFID_val =

b'1\r\n'
Mon Nov

b'turbiditysensor,48.00\r\n'
Mon Nov 30 15:06:27 2020 -- turbiditysensor,48.00

b'turbiditysensor,48.00\r\n'
Mon Nov 30 15:06:28 2020 -- turbiditysensor,48.00



SerialException: read failed: [Errno 6] Device not configured